## Algoritmo de Eliminación de Variables

Este algoritmo es para hacer inferencia en redes. Supongamos que tenemos la factorización de una distribución conjunta 

$$ P(\mathbf{X}) = P(X_1, X_2, \dots, X_N) = \prod_{i=1}^N P(X_i|Par(X_i))$$ 

y una evidenica $ \mathbf{Z}=\mathbf{z} $, donde $\mathbf{Z} \subset \mathbf{X}$ es un subconjunto de las variables del problema y $\mathbf{z}$ son sus valores observados. El objetivo es obtener la distribución de parte de las variables del problema, $\mathbf{W} \subset \mathbf{X}$ dada la evidencia $\mathbf{Z}=\mathbf{z}$. Es decir queremos obtener $P(\mathbf{W}|\mathbf{Z}=\mathbf{z})$. Para ello debemos:
* Reducir los factores que incluyan $\mathbf{Z}$
* Eliminar el resto de variables no incluidas $\mathbf{W}$

$$ P(\mathbf{W}|\mathbf{Z}=\mathbf{z}) = \sum_{X \setminus (W\cup Z)} \frac{P(\mathbf{X}\setminus \mathbf{Z},\mathbf{Z}=\mathbf{z})}{P(\mathbf{Z}=\mathbf{z})} \propto \sum_{X \setminus (W\cup Z)} P(\mathbf{X}\setminus \mathbf{Z},\mathbf{Z}=\mathbf{z}) $$


Algoritmo de eliminación de variables esquemático para un conjunto de factores $\mathbf{\Phi}=\{\Phi_1,\dots,\Phi_N\}$:
1.  Reducir todos los factores que contengan alguna variable de $\mathbf{Z}$ en su dominio, usando la evidencia dada $\mathbf{Z}=\mathbf{z}$
2.  Para cada varible X no en $\mathbf{X} \setminus (\mathbf{W} \cup \mathbf{Z})$, eliminar variable X mediante marginalización:
    1. Hacer el producto de todos los factores que tienen X es su dominio: $\psi = \prod_{\Phi_i \mid X\in Dom(\Phi_i) }\Phi_i$ 
    2. Marginalizar X del factor producto obtenido en A: $\tau = \sum_X \psi$
    3. Actualizar la lista de factores quitando los factores que incluyen X y añadiendo el factor marginalizado $\tau$: $\mathbf{\Phi} = (\mathbf{\Phi} \setminus {\psi}) \cup \tau$
3. Multiplicar factores restantes
4. Renormalizar para obtener una distribución

In [1]:
import numpy as np
import math

In [2]:
# Definamos los factores del problema con el siguiente orden de variables
# Dimensión -> 0  1  2  3  4
# Variable  -> I, D, G, L, S

PI = np.array([0.7, 0.3]).reshape((2,1,1,1,1))
PD = np.array([0.6, 0.4]).reshape((1,2,1,1,1))
PG_ID = np.array([0.3, 0.4, 0.3, 0.05, 0.25, 0.7, 0.9, 0.08, 0.02, 0.5, 0.3, 0.2]).reshape((2,2,3,1,1))
PL_G = np.array([0.1, 0.9, 0.4, 0.6, 0.99, 0.01]).reshape((1,1,3,2,1))
PS_I = np.array([0.95, 0.05, 0.2, 0.8]).reshape((2,1,1,1,2))

def normalize(distribution):
    return distribution/np.sum(distribution)

def reduce(distribution, variables, asignments, normalize_output=True):
    """ This function receives a distribution, 
        a list of indices to variables and 
        a list of the assignements to those variables """
    
    reduced = distribution.copy()
    
    for variable, asignment in zip(variables,asignments):
        reduced = np.swapaxes(reduced, 0, variable)[[asignment]]
        reduced = np.swapaxes(reduced, 0, variable)
        
    return normalize(reduced) if normalize_output else reduced

def marginal(distribution, variables):
    """ Marginalizes the distributions for the given list of variables """
    
    return np.sum(distribution, axis=tuple(variables), keepdims=True)

In [3]:
def VA(factor_list, W, Zs=[], zs=[], order=[]):
    """ Implementar variable elimination algorithm
    
        Entrada:
           * factor_list: lista con los factores a procesar
           * W:           lista de variables en el factor de salida
           * W:          lista de variables observadas
           * zs:          lista de valores de las variables observadas
           * order:       orden en que se procesan las variables. Si no se 
                          indica nada se hacer en orden ascendente
        Salida:
           * Factor con la distribucion conjunta W dada la evidencia
           * El tamaño del factor más grande que se procese
        
    """
    
    factor_list = np.array(factor_list, dtype = object)
    n = factor_list.shape[0]

    # Orden en el que se procesan las variables
    if order != []:
        order += list(set(range(n)) - set(order))
        factor_list = factor_list[order]
    
    
    # Factores reducidos
    for (Z, z) in zip(Zs, zs):
        factor_list = [reduce(factor, [Z], [z]) if factor.shape[Z] > 1 else factor for factor in factor_list]
    
    # Factores a marginalizar
    Xs = set(range(n)) - set(W + Zs)
    
    # Marginalizar producto
    maxsize = 0
    for X in Xs:
        X_psi = [factor for factor in factor_list if factor.shape[X] > 1]
        psi = math.prod(X_psi)
        maxsize = max(maxsize, np.prod(psi.shape))
        tau = marginal(psi, [X])
        factor_list = [factor for factor in factor_list if factor.shape[X] == 1]
        factor_list.append(tau)
        
    # Multiplicar restantes y normalizar 
    return normalize(math.prod(factor_list)), maxsize

In [4]:
# Calcula la distribución P(I)
factor, maxsize = VA([PI, PD, PG_ID, PL_G, PS_I],[0])
assert(np.allclose(np.array([[[[[0.7]]]],[[[[0.3]]]]]),factor))
assert(maxsize==12)

# Si sabemos que la nota del examen es aprobado, ¿Cuál es la prob de inteligencia? 
# P(I|G=g2)
factor, maxsize = VA([PI, PD, PG_ID, PL_G, PS_I],[0],[2],[2])
assert(np.allclose(np.array([[[[[0.92105263]]]],[[[[0.07894737]]]]]), factor))
assert(maxsize==4)

# y si además el examen es difícil
# P(I|G=g2,D=d1)
factor, maxsize = VA([PI, PD, PG_ID, PL_G, PS_I],[0],[1,2],[1,2])
assert(np.allclose(np.array([[[[[0.89090909]]]],[[[[0.10909091]]]]]), factor))
assert(maxsize==4)

In [5]:
# Prob examen: Calcula la distribución: P(D)
factor, maxsize = VA([PI, PD, PG_ID, PL_G, PS_I],[1])
assert(np.allclose(np.array([[[[[0.6]]],[[[0.4]]]]]),factor))
assert(maxsize==24)

# Prob examen | nota aprobado: P(D|G=g2)
factor, maxsize = VA([PI, PD, PG_ID, PL_G, PS_I],[1],[2],[2])
assert(np.allclose(np.array([[[[[0.37070938]]],[[[0.62929062]]]]]),factor))
assert(maxsize==8)

# Probabilidad de examen difícil D=d1|G=g2,S=s1?
factor, maxsize = VA([PI, PD, PG_ID, PL_G, PS_I],[1],[2,4],[2,1])
assert(np.allclose(np.array([[[[[0.24044002]]],[[[0.75955998]]]]]),factor))
assert(maxsize==4)

In [6]:
# Si no se conoce G, ¿Influye la nota de selectividad en la dificultad del examen?
# dif examen
print(VA([PI, PD, PG_ID, PL_G, PS_I],[1]))

# dif examen si sat=1
print(VA([PI, PD, PG_ID, PL_G, PS_I],[1],[4],[1])) # No cambia

# Ahora sabiendo que nota es aprobado
print(VA([PI, PD, PG_ID, PL_G, PS_I],[1],[2],[2])) 
print(VA([PI, PD, PG_ID, PL_G, PS_I],[1],[2,4],[2,1])) # Sí cambia

(array([[[[[0.6]]],


        [[[0.4]]]]]), 24)
(array([[[[[0.6]]],


        [[[0.4]]]]]), 12)
(array([[[[[0.37070938]]],


        [[[0.62929062]]]]]), 8)
(array([[[[[0.24044002]]],


        [[[0.75955998]]]]]), 4)
